In [1]:

import yaml
import json
import os

from neo4j import GraphDatabase




In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [3]:

driver = GraphDatabase.driver(PARAM["neo4j_url"], auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))

records, summary, keys = driver.execute_query(f"""
    CALL apoc.meta.schema()
    YIELD value RETURN value;
    """,
    database_="neo4j",
)
# Loop through results and do something with them
for record in records:
    schema = record.data()["value"]
    json_schema = json.dumps(record.data()["value"])
    #print (json_schema)


In [4]:
schema.keys()

dict_keys(['Condition', 'HAS_MORPHOLOGY', 'Category', 'Site', 'Trial', 'IS_CARRIED_OUT_BY', 'Morphology', 'FOCUSES_ON', 'IS_FOUND_AT_SITE', 'Institution', 'BELONGS_TO'])

In [7]:
schema

{'Condition': {'count': 23,
  'labels': [],
  'properties': {'SNOMEDCT': {'unique': True,
    'indexed': True,
    'type': 'STRING',
    'existence': False},
   'name': {'unique': False,
    'indexed': False,
    'type': 'STRING',
    'existence': False},
   'UMLS': {'unique': False,
    'indexed': False,
    'type': 'STRING',
    'existence': False}},
  'type': 'node',
  'relationships': {'HAS_MORPHOLOGY': {'count': 0,
    'direction': 'out',
    'labels': ['Morphology'],
    'properties': {}},
   'FOCUSES_ON': {'count': 100,
    'direction': 'in',
    'labels': ['Trial'],
    'properties': {}},
   'IS_FOUND_AT_SITE': {'count': 0,
    'direction': 'out',
    'labels': ['Site'],
    'properties': {}},
   'BELONGS_TO': {'count': 0,
    'direction': 'out',
    'labels': ['Category'],
    'properties': {}}}},
 'HAS_MORPHOLOGY': {'count': 14, 'properties': {}, 'type': 'relationship'},
 'Category': {'count': 27,
  'labels': [],
  'properties': {'fsn': {'unique': False,
    'indexed': False,

In [5]:
schema['Condition']['type']

'node'

In [6]:
schema['Condition']

{'count': 23,
 'labels': [],
 'properties': {'SNOMEDCT': {'unique': True,
   'indexed': True,
   'type': 'STRING',
   'existence': False},
  'name': {'unique': False,
   'indexed': False,
   'type': 'STRING',
   'existence': False},
  'UMLS': {'unique': False,
   'indexed': False,
   'type': 'STRING',
   'existence': False}},
 'type': 'node',
 'relationships': {'HAS_MORPHOLOGY': {'count': 0,
   'direction': 'out',
   'labels': ['Morphology'],
   'properties': {}},
  'FOCUSES_ON': {'count': 100,
   'direction': 'in',
   'labels': ['Trial'],
   'properties': {}},
  'IS_FOUND_AT_SITE': {'count': 0,
   'direction': 'out',
   'labels': ['Site'],
   'properties': {}},
  'BELONGS_TO': {'count': 0,
   'direction': 'out',
   'labels': ['Category'],
   'properties': {}}}}

In [15]:
#Condition: {SNOMEDCT: STRING}
schema_dict = {}
node_index = {}

relation_A_B = {}

for key in schema.keys():
    if schema[key]['type'] == 'node':
        schema_dict[key] = {}

        for p in schema[key]['properties'].keys():
            schema_dict[key][p] = schema[key]['properties'][p]['type']

            if schema[key]['properties'][p]['unique'] == True:
                node_index[key] = p
        

        for r in schema[key]['relationships'].keys():
            if schema[key]['relationships'][r]["direction"] == "out":
                relation_A_B[r] = (key, schema[key]['relationships'][r]["labels"][0])

In [28]:
schema_dict

{'Condition': {'SNOMEDCT': 'STRING', 'name': 'STRING', 'UMLS': 'STRING'},
 'Category': {'fsn': 'STRING', 'SNOMEDCT': 'STRING', 'name': 'STRING'},
 'Site': {'fsn': 'STRING', 'SNOMEDCT': 'STRING', 'name': 'STRING'},
 'Trial': {'min_age': 'STRING',
  'healthy_volunteers': 'STRING',
  'study_results': 'STRING',
  'outcome_measures': 'LIST',
  'criteria': 'STRING',
  'status': 'STRING',
  'max_age': 'STRING',
  'study_type': 'STRING',
  'url': 'STRING',
  'title': 'STRING',
  'locations': 'LIST',
  'description': 'STRING',
  'name': 'STRING',
  'phases': 'LIST',
  'gender': 'STRING',
  'enrollment': 'STRING',
  'start_date': 'STRING',
  'sampling_method': 'STRING'},
 'Morphology': {'fsn': 'STRING', 'SNOMEDCT': 'STRING', 'name': 'STRING'},
 'Institution': {'name': 'STRING', 'type': 'STRING'}}

In [13]:
node_index

{'Condition': 'SNOMEDCT',
 'Category': 'SNOMEDCT',
 'Site': 'SNOMEDCT',
 'Trial': 'name',
 'Morphology': 'SNOMEDCT',
 'Institution': 'name'}

In [16]:
relation_A_B

{'HAS_MORPHOLOGY': ('Condition', 'Morphology'),
 'IS_FOUND_AT_SITE': ('Condition', 'Site'),
 'BELONGS_TO': ('Condition', 'Category'),
 'IS_CARRIED_OUT_BY': ('Trial', 'Institution'),
 'FOCUSES_ON': ('Trial', 'Condition')}

In [29]:
schema['Condition']['properties']

{'SNOMEDCT': {'unique': True,
  'indexed': True,
  'type': 'STRING',
  'existence': False},
 'name': {'unique': False,
  'indexed': False,
  'type': 'STRING',
  'existence': False},
 'UMLS': {'unique': False,
  'indexed': False,
  'type': 'STRING',
  'existence': False}}

In [30]:
for label in schema.keys():
    if schema[label]['type'] == 'node':
        print (label)

Condition
Category
Site
Trial
Morphology
Institution


In [31]:
# variable_name = "t"
# output_directory = "tsv"

# for node_type in schema.keys():
#     if schema[node_type]['type'] == 'node':

#         records, summary, keys = driver.execute_query(f"""
#             MATCH ({variable_name}:{node_type})
#             RETURN {variable_name}
#             """,
#             database_="neo4j",
#         )
        
#         header = list(schema[node_type]['properties'].keys())
#         content = "\t".join(header) + "\n"
#         for record in records:
#             result = record.data()[f"{variable_name}"]
            
#             for h in header:
#                 if h in result:
#                     content += str(result[h]) + "\t"
#                 else:
#                     content += "\t"
#             content = content[:-1] + "\n"

#         with open(os.path.join(output_directory, f"{node_type}.tsv"), 'w') as f:
#             f.write(content)


In [9]:
for node_type in schema.keys():
    if schema[node_type]['type'] == 'relationship':
        print (node_type, schema[node_type])

HAS_MORPHOLOGY {'count': 14, 'properties': {}, 'type': 'relationship'}
IS_CARRIED_OUT_BY {'count': 120, 'properties': {}, 'type': 'relationship'}
FOCUSES_ON {'count': 100, 'properties': {}, 'type': 'relationship'}
IS_FOUND_AT_SITE {'count': 12, 'properties': {}, 'type': 'relationship'}
BELONGS_TO {'count': 53, 'properties': {}, 'type': 'relationship'}


In [18]:
variable_name = "t"
output_directory = "json"

for relation_type in relation_A_B.keys():
    A = relation_A_B[relation_type][0]
    B = relation_A_B[relation_type][1]
    
    A_index = node_index[A]
    B_index = node_index[B]


    records, summary, keys = driver.execute_query(f"""
            MATCH (A_:{A}) -[r:{relation_type}]-> (B_:{B})
            RETURN A_.{A_index}, B_.{B_index}
            """,
            database_="neo4j",
        )

    content = ""
        #header = list(schema[node_type]['properties'].keys())
        #content = "\t".join(header) + "\n"
    for record in records:
            
        from_ = record.data()[f"A_.{A_index}"]
        to_ = record.data()[f"B_.{B_index}"]

        content += json.dumps({"from": from_, "to": to_}) + "\n"
    
    with open(os.path.join(output_directory, f"{relation_type}.json"), 'w') as f:
            f.write(content)

In [ ]:
output_directory = "json"

for node_type in schema.keys():
    if schema[node_type]['type'] == 'node':

        records, summary, keys = driver.execute_query(f"""
            MATCH ({variable_name}:{node_type})
            RETURN {variable_name}
            """,
            database_="neo4j",
        )
        
        content = ""
        #header = list(schema[node_type]['properties'].keys())
        #content = "\t".join(header) + "\n"
        for record in records:
            
            content += json.dumps(record.data()[variable_name]) + "\n"

        with open(os.path.join(output_directory, f"{node_type}.json"), 'w') as f:
            f.write(content)

In [19]:
bucket_name = PARAM["gs_bucket"]

In [34]:
#os.system(f'{PARAM["gsutil_path"]}/gsutil cp -r tsv gs://{bucket_name}/')

In [20]:
os.system(f'{PARAM["gsutil_path"]}/gsutil cp -r json gs://{bucket_name}/')

Copying file://json/Condition.json [Content-Type=application/json]...
Copying file://json/Category.json [Content-Type=application/json]...            
Copying file://json/Site.json [Content-Type=application/json]...                
Copying file://json/Trial.json [Content-Type=application/json]...               
- [4 files][292.8 KiB/292.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://json/Morphology.json [Content-Type=application/json]...
Copying file://json/Institution.json [Content-Type=application/json]...         
Copying file://json/HAS_MORPHOLOGY.json [Content-Type=application/json]...      
Copying file://json/IS_FOUND_AT_SITE.json [Content-Type=application/json]...    
Copying file://json/BELONGS_TO.json [Con

0

In [22]:
from google.cloud import bigquery
from google.cloud import storage

In [23]:
bq_client = bigquery.Client()
bigquery_project = PARAM["bigquery_project"]
bigquery_dataset = PARAM["bigquery_dataset"]
bq_client.create_dataset(dataset=bigquery_dataset, exists_ok=True)

Dataset(DatasetReference('vertex-ai-399007', 'neo4j'))

In [24]:
bucket_client = storage.Client()
bucket = bucket_client.bucket(bucket_name)

In [39]:
# for f in bucket.list_blobs(prefix='tsv'):
    
#     full_path = f"gs://{bucket_name}/{f.name}"

#     filename = f.name.split("/")[1]
#     nodename = filename.split(".")[0]
#     print (full_path, nodename)

In [25]:
for f in bucket.list_blobs(prefix='json'):
#for f in bucket.list_blobs(prefix='tsv'):
    
    full_path = f"gs://{bucket_name}/{f.name}"

    filename = f.name.split("/")[1]
    nodename = filename.split(".")[0]
    print (full_path, nodename)

    #schema_setting = [bigquery.SchemaField(property_, schema_dict[nodename][property_]) for property_ in schema_dict[nodename].keys()]
    schema_setting = []

    # for property_ in schema_dict[nodename].keys():
    #     type_ = schema_dict[nodename][property_]
    #     if type_ != "LIST":
    #         schema_setting.append(bigquery.SchemaField(property_, type_))
    #     else:
 
    #         schema_setting.append(bigquery.SchemaField(property_, "STRING", mode="REPEATED"))

    #print (schema_setting)
    job_config = bigquery.LoadJobConfig(
    #schema=schema_setting,
    autodetect=True,
    #skip_leading_rows=1,
    #field_delimiter="\t",
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    )

    table_id = f"{bigquery_project}.{bigquery_dataset}.{nodename}"

    load_job = bq_client.load_table_from_uri(
        full_path, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = bq_client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))

gs://neo4j-bigquery-project/json/BELONGS_TO.json BELONGS_TO
Loaded 53 rows.
gs://neo4j-bigquery-project/json/Category.json Category
Loaded 27 rows.
gs://neo4j-bigquery-project/json/Condition.json Condition
Loaded 23 rows.
gs://neo4j-bigquery-project/json/FOCUSES_ON.json FOCUSES_ON
Loaded 100 rows.
gs://neo4j-bigquery-project/json/HAS_MORPHOLOGY.json HAS_MORPHOLOGY
Loaded 14 rows.
gs://neo4j-bigquery-project/json/IS_CARRIED_OUT_BY.json IS_CARRIED_OUT_BY
Loaded 120 rows.
gs://neo4j-bigquery-project/json/IS_FOUND_AT_SITE.json IS_FOUND_AT_SITE
Loaded 12 rows.
gs://neo4j-bigquery-project/json/Institution.json Institution
Loaded 97 rows.
gs://neo4j-bigquery-project/json/Morphology.json Morphology
Loaded 12 rows.
gs://neo4j-bigquery-project/json/Site.json Site
Loaded 2 rows.
gs://neo4j-bigquery-project/json/Trial.json Trial
Loaded 73 rows.
